This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/development/sdk-install-guides/databricks-azure-sdk-install).

## Installing the Wallaroo SDK into  Workspace

Organizations that use Azure Databricks for model training and development can deploy models to Wallaroo through the [Wallaroo SDK](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/).  The following guide is created to assist users with installing the Wallaroo SDK, setting up authentication through Azure Databricks, and making a standard connection to a Wallaroo instance through Azure Databricks Workspace.

These instructions are based on the on the [Wallaroo SSO for Microsoft Azure](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-sso-authentication/wallaroo-sso-azure/) and the [Connect to Wallaroo](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/) guides.

This tutorial provides the following:

* `ccfraud.onnx`:  A pretrained model from the Machine Learning Group's demonstration on [Credit Card Fraud detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).
* Sample inference test data:
  * `ccfraud_high_fraud.json`: Test input file that returns a high likelihood of credit card fraud.
  * `ccfraud_smoke_test.json`: Test input file that returns a low likelihood of credit card fraud.
  * `cc_data_1k.json`:  Sample input file with 1,000 records.
  * `cc_data_10k.json`:  Sample input file with 10,000 records.

To use the Wallaroo SDK within Azure Databricks Workspace, a virtual environment will be used.  This will set the necessary libraries and specific Python version required.

## Prerequisites

The following is required for this tutorial:

* A Wallaroo instance version 2022.4 or later with [External Inference URls enabled](https://docs.wallaroo.ai/wallaroo-tutorials/wallaroo-tutorial-features/wallaroo-model-endpoints/wallaroo-external-inference-tutorial/).
* An Azure Databricks workspace with a cluster

## General Steps

For our example, we will perform the following:

* Wallaroo SDK Install
  * Install the Wallaroo SDK into the Azure Databricks cluster.
  * Install the Wallaroo Python SDK.
  * Connect to a remote Wallaroo instance.  This instance is configured to use the standard Keycloak service.
* Wallaroo SDK from Azure Databricks Workspace (Optional)
  * The following steps are used to demonstrate using the Wallaroo SDK in an Azure Databricks Workspace environment.  The entire tutorial can be found on the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/sdk-install-guides/azure-ml-sdk-install).
    * Create a workspace for our work.
    * Upload the CCFraud model.
    * Create a pipeline that can ingest our submitted data, submit it to the model, and export the results
    * Run a sample inference through our pipeline by loading a file
    * Undeploy the pipeline and return resources back to the Wallaroo instance's Kubernetes environment.

## Install Wallaroo SDK

### Add Wallaroo SDK to Cluster

To install the Wallaroo SDK in a Azure Databricks environment:

1. From the Azure Databricks dashboard, select **Computer**, then the cluster to use.
1. Select **Libraries**.
1. Select **Install new**.
1. Select **PyPI**.  In the **Package** field, enter the current version of the [Wallaroo SDK](https://pypi.org/project/wallaroo/).  It is recommended to specify the version, which as of this writing is `wallaroo==2022.4.0`.
1. Select **Install**.

Once the **Status** shows **Installed**, it will be available in Azure Databricks notebooks and other tools that use the cluster.

### Add Tutorial Files

The following instructions can be used to upload this tutorial and it's files into Databricks.  Depending on how your Azure Databricks is configured and your organizations standards, there are multiple ways of uploading files to your Azure Databricks environment.  The following example is used for the tutorial and makes it easy to reference data files from within this Notebook.  Adjust based on your requirements.

* **IMPORTANT NOTE**:  Importing a repo from a Git repository may not convert the included Jupyter Notebooks into the Databricks format.  This method 

1. From the Azure Databricks dashboard, select **Repos**.
1. Select where to place the repo, then select **Add Repo**.
 
    ![](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure-databricks-select-add-repo.png)

1. Set the following:
 
    ![](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure-databricks-add-repo-settings.png)

    1. **Create repo by cloning a Git repository**:  Uncheck
    1. **Repository name**:  Set any name based on the Databricks standard (no spaces, etc).
    1. Select **Create Repo**.
1. Select the new tutorial, then from the repo menu dropdown, select **Import**.
 
    ![](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure-databricks-repo-select-import.png)

1. Select the files to upload.  For this example, the following files are uploaded:

    ![](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure-databricks-repo-import-files.png)

    1. `ccfraud.onnx`:  A pretrained model from the Machine Learning Group's demonstration on [Credit Card Fraud detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).
    1. Sample inference test data:
        1. `ccfraud_high_fraud.json`: Test input file that returns a high likelihood of credit card fraud.
        1. `ccfraud_smoke_test.json`: Test input file that returns a low likelihood of credit card fraud.
        1. `cc_data_1k.json`:  Sample input file with 1,000 records.
        1. `cc_data_10k.json`:  Sample input file with 10,000 records.
    1. `install-wallaroo-sdk-databricks-azure-guide.ipynb`: This notebook.
1. Select **Import**.

The Jupyter Notebook can be opened from this new Azure Databricks repository, and relative files it references will be accessible with the exceptions listed below.

Zip files added via the method above are automatically decompressed, so can not be used as model files.  For example, tensor based models such as the [Wallaroo Aloha Demo](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/aloha).  Zip files can be uploaded using DBFS and used through the following process:

To upload model files to Azure Databricks using DBFS:

1. From the Azure Databricks dashboard, select **Data**.
1. Select **Add->Add data**.

    ![](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure_databricks_select_add_data.png)
 
1. Select **DBFS**.

    ![](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure_databricks_select_dbfs.png)

1. Select **Upload File** and enter the following:
 
    ![](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure_databricks_upload_files.png) 

    1. **DBFS Target Directory** (*Optional*): Optional step:  Set the directory where the files will be uploaded.
1. Select the files to upload.  Note that each file will be given a location and they can be access with `/dbfs/PATH`.  For example, the file `alohacnnlstm.zip` uploaded to the directory `aloha` would be referenced with `/dbfs/FileStore/tables/aloha/alohacnnlstm.zip

## Sample Wallaroo Connection

With the Wallaroo Python SDK installed, remote commands and inferences can be performed through the following steps.

### Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.

This is accomplished using the `wallaroo.Client(api_endpoint, auth_endpoint, auth_type command)` command that connects to the Wallaroo instance services.

The `Client` method takes the following parameters:

* **api_endpoint** (*String*): The URL to the Wallaroo instance API service.
* **auth_endpoint** (*String*): The URL to the Wallaroo instance Keycloak service.
* **auth_type command** (*String*): The authorization type.  In this case, `SSO`.

The URLs are based on the Wallaroo Prefix and Wallaroo Suffix for the Wallaroo instance.  For more information, see the [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).  In the example below, replace "YOUR PREFIX" and "YOUR SUFFIX" with the Wallaroo Prefix and Suffix, respectively.  In the example below, replace "YOUR PREFIX" and "YOUR SUFFIX" with the Wallaroo Prefix and Suffix, respectively.

Once run, the `wallaroo.Client` command provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.

Depending on the configuration of the Wallaroo instance, the user will either be presented with a login request to the Wallaroo instance or be authenticated through a broker such as Google, Github, etc.  To use the broker, select it from the list under the username/password login forms.  For more information on Wallaroo authentication configurations, see the [Wallaroo Authentication Configuration Guides](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-sso-authentication/).

![Wallaroo Login](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/databricks-azure-sdk-guide/azure-initial-login.png)

Once authenticated, the user will verify adding the device the user is establishing the connection from.  Once both steps are complete, then the connection is granted.

![Device Registration](./images/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guides/wallaroo-device-access.png)

The connection is stored in the variable `wl` for use in all other Wallaroo calls.

Replace `YOUR PREFIX` and `YOUR SUFFIX` with the DNS prefix and suffix for the Wallaroo instance.  For more information, see the [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [ ]:
import wallaroo
from wallaroo.object import EntityNotFoundError

In [ ]:
# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR SUFFIX"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Create the Workspace

We will create a workspace to work in and call it the `databricksazuresdkworkspace`, then set it as current workspace environment.  We'll also create our pipeline in advance as `databricksazuresdkpipeline`.

* **IMPORTANT NOTE**:  For this example, the CCFraud model is stored in the file `ccfraud.onnx` and is referenced from a relative link.  For platforms such as Databricks, the files may need to be in a universal file format.  For those, the example file location below may be:

`model_file_name = '/dbfs/FileStore/tables/aloha/alohacnnlstm.zip`

Adjust file names and locations based on your requirements.

In [ ]:
workspace_name = 'databricksazuresdkworkspace'
pipeline_name = 'databricksazuresdkpipeline'
model_name = 'ccfraudmodel'
model_file_name = './ccfraud.onnx'

In [ ]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [ ]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline

name,databricksazuresdkpipeline
created,2023-02-07 15:55:40.574745+00:00
last_updated,2023-02-07 15:55:40.574745+00:00
deployed,(none)
tags,
versions,ca6d6ea6-2e45-4795-9253-5c40e8483dc9
steps,


We can verify the workspace is created the current default workspace with the `get_current_workspace()` command.

In [ ]:
wl.get_current_workspace()

Out[6]: {'name': 'databricksazuresdkworkspace', 'id': 8, 'archived': False, 'created_by': '3547815c-b48d-4e69-bfbd-fff9d525c5d7', 'created_at': '2023-02-07T15:55:39.548497+00:00', 'models': [], 'pipelines': [{'name': 'databricksazuresdkpipeline', 'create_time': datetime.datetime(2023, 2, 7, 15, 55, 40, 574745, tzinfo=tzutc()), 'definition': '[]'}]}

### Upload the Models

Now we will upload our model.

**IMPORTANT NOTE**:  If using DBFS, use the file path format such as `/dbfs/FileStore/shared_uploads/YOURWORKSPACE/file` format rather than the `dbfs:` format.

In [ ]:
model = wl.upload_model(model_name, model_file_name).configure()
model

Out[15]: {'name': 'ccfraudmodel', 'version': 'ccb488dd-36ed-4aaf-99cf-9a16bd3654db', 'file_name': 'ccfraud.onnx', 'image_path': None, 'last_update_time': datetime.datetime(2023, 2, 7, 16, 1, 0, 303545, tzinfo=tzutc())}

### Deploy a Model

Now that we have a model that we want to use we will create a deployment for it. 

To do this, we'll create our pipeline that can ingest the data, pass the data to our CCFraud model, and give us a final output.  We'll call our pipeline `databricksazuresdkpipeline`, then deploy it so it's ready to receive data.  The deployment process usually takes about 45 seconds.

In [ ]:
pipeline.add_model_step(model)

name,databricksazuresdkpipeline
created,2023-02-07 15:55:40.574745+00:00
last_updated,2023-02-07 15:57:24.803281+00:00
deployed,True
tags,
versions,"971f9db7-1b73-4e72-8cdb-cfa2d5a9ddd7, 6c1028c4-3ca7-47b0-b3a6-834d12b57fc9, ca6d6ea6-2e45-4795-9253-5c40e8483dc9"
steps,ccfraudmodel


In [ ]:
pipeline.deploy()

name,databricksazuresdkpipeline
created,2023-02-07 15:55:40.574745+00:00
last_updated,2023-02-07 16:04:35.891487+00:00
deployed,True
tags,
versions,"091eed0f-8984-4753-9316-0fbbf68bb398, bb701da9-440b-4ce6-8b92-36446347e85c, 971f9db7-1b73-4e72-8cdb-cfa2d5a9ddd7, 6c1028c4-3ca7-47b0-b3a6-834d12b57fc9, ca6d6ea6-2e45-4795-9253-5c40e8483dc9"
steps,ccfraudmodel


We can verify that the pipeline is running and list what models are associated with it.

In [ ]:
pipeline.status()

Out[23]: {'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.2.34',
   'name': 'engine-754b5c457d-5c4pc',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'databricksazuresdkpipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'ccfraudmodel',
      'version': 'ccb488dd-36ed-4aaf-99cf-9a16bd3654db',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.0.29',
   'name': 'engine-lb-74b4969486-mslkt',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

## Interferences

### Infer 1 row

Now that the pipeline is deployed and our CCfraud model is in place, we'll perform a smoke test to verify the pipeline is up and running properly.  We'll use the `infer_from_file` command to load a single transaction and determine if it is flagged for fraud.  If it returns correctly, a small value should be returned indicating a low likelihood that the transaction was fraudulent.

In [ ]:
result = pipeline.infer_from_file("./ccfraud_smoke_test.json")


In [ ]:
result[0].data()

Out[25]: [array([[0.00149742]])]

### Batch Inference

Now that our smoke test is successful, let's really give it some data.  We'll use the `cc_data_1k.json` file that contains 1,000 inferences to be performed.

In [ ]:
result = pipeline.infer_from_file("./cc_data_1k.json")
result

Out[26]: [InferenceResult({'check_failures': [],
  'elapsed': 245003,
  'model_name': 'ccfraudmodel',
  'model_version': 'ccb488dd-36ed-4aaf-99cf-9a16bd3654db',
  'original_data': {'tensor': [[-1.060329750089797,
                                2.354496709462385,
                                -3.563878832646437,
                                5.138734892618555,
                                -1.23084570186641,
                                -0.7687824607744093,
                                -3.588122810891446,
                                1.888083766259287,
                                -3.2789674273886593,
                                -3.956325455353324,
                                4.099343911805088,
                                -5.653917639476211,
                                -0.8775733373342495,
                                -9.131571191990632,
                                -0.6093537872620682,
                                -3.748027677256424,
        

## Undeploy Pipeline

When finished with our tests, we will undeploy the pipeline so we have the Kubernetes resources back for other tasks.  Note that if the deployment variable is unchanged pipeline.deploy() will restart the inference engine in the same configuration as before.

In [ ]:
pipeline.undeploy()